<a href="https://colab.research.google.com/github/T0McKay/HIS406_Assignment2_DiabetesPrediction_TheoMcKay/blob/main/diabetes_predictor_Theo_McKay.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#AI + ML Assignment 2 - Diabetes Predictor

##Data Collection and Information Gathering

The Prima Indians Diabetes Dataset will be used for training and validation. This has 9 columns and 768 rows.

[link text](https://)Link: https://www.kaggle.com/datasets/uciml/pima-indians-diabetes-database

In [12]:
import kagglehub
import pandas as pd

# Download latest version
path = kagglehub.dataset_download("uciml/pima-indians-diabetes-database")

# Assign variable to represent dataset
df = pd.read_csv("/kaggle/input/pima-indians-diabetes-database/diabetes.csv")

Using Colab cache for faster access to the 'pima-indians-diabetes-database' dataset.


###Information Gathering

In [13]:
# displays first 5 rows
print(df.head())
# displays information such as column labels, and null values.
print(df.info())
# displays statistic description for each row, including mean, variance and standard deviation.
print(df.describe())

   Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0            6      148             72             35        0  33.6   
1            1       85             66             29        0  26.6   
2            8      183             64              0        0  23.3   
3            1       89             66             23       94  28.1   
4            0      137             40             35      168  43.1   

   DiabetesPedigreeFunction  Age  Outcome  
0                     0.627   50        1  
1                     0.351   31        0  
2                     0.672   32        1  
3                     0.167   21        0  
4                     2.288   33        1  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768

In [14]:
# gets all  numeric features
numerical = [feature for feature in df.columns if df[feature].dtype != 'O']
print('Number of Numerical Features :', len(numerical))

# gets all categorical features
categorical = [feature for feature in df.columns if df[feature].dtype == 'O']
print('Number of Categorical Features :', len(categorical))

# displays number of discrete features
discrete = [feature for feature in numerical if len(df[feature].unique())<=25]
print('Number of Discrete Features :',len(discrete))

# displays number of continuous features
continuous = [feature for feature in numerical if feature not in discrete]
print('Number of Continuous Features :',len(continuous))

Number of Numerical Features : 9
Number of Categorical Features : 0
Number of Discrete Features : 2
Number of Continuous Features : 7


####Insight from "Information Gathering" :

All categories have numerical values.

Two of these are discrete(Pregnancies and Outcome), two are continuous.

##Data Preprocessing

###Data Cleaning

####Missing Value Check

In [15]:
print(df.isnull().sum()) # shows there are no null values in dataset, so no fix needed

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64


####Duplicate Records Check and Delete

In [16]:
# shows no duplicates that need to get deleted
df.duplicated().sum()
# deletes duplications
df = df.drop_duplicates()

### Feature Validation

####Invalid Zero Values
Some columns can have zeros as a value where its not possible. This needs to be checked and changed.

Columns include: Glucose, BloodPressure, SkinThickness, Insulin and BMI

In [17]:
# list of columns that could have invalid zero values
checkForInvalid = [
    'Glucose',
    'BloodPressure',
    'SkinThickness',
    'Insulin',
    'BMI'
]

# checks for invalid zeros
(df[checkForInvalid] == 0).sum()

,0
Glucose,5
BloodPressure,35
SkinThickness,227
Insulin,374
BMI,11


#####Median Imputation
Invalid zeros in the Insulin column will be replaced with the median (middle value) as the mean may be scewed by extreme values.

In [18]:
#gets median for Insulin and replaces zeros
medianInsulin = df['Insulin'].median()
df['Insulin'] = df['Insulin'].replace(0, medianInsulin)

#####Mean Imputation
This will be used to replace the invalid zeros in columns Glucose, BloodPressure, SkinThickness and BMI.

This is because they are all continuous and normally distributed.

In [19]:
# remaining list of columns
remainingChecks = [
    'Glucose',
    'BloodPressure',
    'SkinThickness',
    'BMI'
]

for column in remainingChecks:
    median = df[column].median()
    df[column] = df[column].replace(0, median)

#####Check No Invalid Zeros

In [20]:
(df[checkForInvalid] == 0).sum()

,0
Glucose,0
BloodPressure,0
SkinThickness,0
Insulin,0
BMI,0


###Train/Test Split
The dataset will be split 80/20.

80% will be used to train the model. 20% will be used to test the model.

In [21]:
from sklearn.model_selection import train_test_split

In [22]:
x = df.drop('Outcome', axis=1)
y = df['Outcome']

In [23]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)